# Azure AI Search with NVIDIA NIM மற்றும் LlamaIndex ஒருங்கிணைப்பு

In this notebook, we'll demonstrate how to leverage NVIDIA's AI models and LlamaIndex to create a powerful Retrieval-Augmented Generation (RAG) pipeline. We'll use NVIDIA's LLMs and embeddings, integrate them with Azure AI Search as the vector store, and perform RAG to enhance search quality and efficiency.

## நன்மைகள்
- **அளவீடாக்கம்**: NVIDIA இன் பெரிய மொழி மாடல்களையும் Azure AI Search ஐ பரவலாகவும் செயல்திறனாகவும் மீட்பதற்காக பயன்படுத்துங்கள்.
- **செலவு செயல்திறன்**: திறனான வெக்டர் சேமிப்பும் ஹைபிரிட் தேடல் நுட்பங்களும் மூலம் தேடல் மற்றும் மீட்பை ஒழுங்குபடுத்துங்கள்.
- **உயர்தர செயல்திறன்**: சக்திவாய்ந்த LLMகளை வெக்டர் செய்யப்பட்ட தேடலுடன் இணைத்தல் மூலம் வேகமாகவும் துல்லியமாகவும் பதில்களை பெறுங்கள்.
- **தரம்**: தொடர்புடைய மீட்டெடுக்கப்பட்ட ஆவணங்களால் LLM பதில்களை நிலைநாட்டுவதன் மூலம் உயர்ந்த தேடல் தரத்தை பராமரிக்கவும்.

## தேவையானவை
- 🐍 Python 3.9 அல்லது அதற்கு மேல்
- 🔗 [Azure AI Search சேவை](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA NIM மைக்ரோசேவைகள் மூலம் NVIDIA இன் LLMகளுக்கும் Embeddings க்குமான அணுகலுக்கு NVIDIA API விசை

## உள்ள அம்சங்கள்
- ✅ NVIDIA LLM ஒருங்கிணைப்பு (நாம் பயன்படுத்தப் போகிறோம் [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA Embeddings (நாம் பயன்படுத்தப் போகிறோம் [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Azure AI Search மேம்பட்ட பெறுதல் முறைகள்
- ✅ LlamaIndex உடன் ஆவணக் குறியீடு
- ✅ Azure AI Search மற்றும் LlamaIndex உடன் NVIDIA LLMகளை பயன்படுத்தி RAG

தொடங்குவோம்!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## நிறுவல் மற்றும் தேவைகள்

Python பதிப்பு >3.10 பயன்படுத்தி ஒரு Python சூழலை உருவாக்கவும்.

## தொடக்கம்!


தொடங்குவதற்கு, NVIDIA AI Foundation மாதிரிகளைப் பயன்படுத்த, உங்களுக்கு ஒரு `NVIDIA_API_KEY` தேவை:
1) ஒரு இலவச கணக்கை [NVIDIA](https://build.nvidia.com/explore/discover) உடன் உருவாக்குங்கள்.
2) தேர்ந்தெடுத்த மாதிரியை கிளிக் செய்யவும்.
3) Input கீழ் Python தாவலைத் தேர்ந்தெடுத்து, **API விசையைப் பெறவும்** கிளிக் செய்து பின்னர் **சாவியை உருவாக்கவும்** கிளிக் செய்யவும்.
4) உருவாக்கப்பட்ட சாவியை நகலெடுத்து NVIDIA_API_KEY ஆக சேமிக்கவும். அதிலிருந்து, நீங்கள் endpoints-களை அணுகலாம்.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG உதாரணம் LLM மற்றும் எம்பெடிங் பயன்படுத்தி
### 1) LLM ஐ ஆரம்பிக்கவும்
`llama-index-llms-nvidia`, எனப்படும் NVIDIA-இன் LLM இணைப்பி, NVIDIA API catalog இல் கிடைக்கும் பொருந்தக்கூடிய மாதிரிகளுடன் இணைந்து அவற்றிலிருந்து உருவாக்கங்களை செய்ய அனுமதிக்கிறது. உரையாடல் முடிப்பு மாதிரிகளின் பட்டியலுக்கு இங்கேப் பார்க்கவும்: https://build.nvidia.com/search?term=Text-to-Text

இங்கே நாம் **mixtral-8x7b-instruct-v0.1** ஐப் பயன்படுத்துவோம்


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) எம்பெடிங்கை ஆரம்பிக்கவும்
`llama-index-embeddings-nvidia`, NVIDIA-வின் எம்பெடிங் கனெக்டர் என்றும் அழைக்கப்படுகிறது, NVIDIA API பட்டியலில் கிடைக்கும் பொருத்தமான மாதிரிகளுடன் இணைக்கவும் அவற்றிலிருந்து உருவாக்கவும் இது உங்களுக்கு அனுமதிக்கிறது. We selected `nvidia/nv-embedqa-e5-v5` as the embedding model. See here for a list of text embedding models: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) ஒரு Azure AI Search வெக்டர் ஸ்டோர் உருவாக்கவும்


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

### 4) ஆவணங்களை ஏற்று, பகுதியாகப் பிரித்து மற்றும் பதிவேற்றவும்


In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) உங்கள் தரவுகளைப் பற்றிய கேள்விகளை கேட்க ஒரு கேள்வி இயந்திரத்தை உருவாக்கவும்

இங்கே Azure AI Search இல் தூய வெக்டர் தேடலைப் பயன்படுத்தி, பதிலை எங்கள் LLM (Phi-3.5-MOE)க்குத் தரவுதள ஆதாரமாக இணைக்கும் ஒரு வினா கொடுக்கப்பட்டுள்ளது.


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

இங்கே Azure AI Search இல் ஹைப்ரிட் தேடலைப் பயன்படுத்தும் ஒரு வினவல் உள்ளது.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### வெக்டர் தேடல் பகுப்பாய்வு
LLM பதில் மூல உரையில் ரஷ்யாவின் பங்கு சந்தையைப் பற்றிச் சொல்லப்பட்ட முக்கிய பொருளாதார விளைவுகளை துல்லியமாகப் பதிவுசெய்கிறது. குறிப்பாக, இது ரஷ்ய பங்கு சந்தை významமிக்க வீழ்ச்சியை அனுபவித்து அதன் மதிப்பிலிருந்து 40% இழந்தது மற்றும் தொடர்ந்துள்ள சூழ்நிலையால் வர்த்தகம் நிறுத்தப்பட்டதாக குறிப்பிடுகிறது. இந்த பதில் மூலத்தில் வழங்கப்பட்ட தகவலுடன் நன்கு ஒத்துப்போகிறது, அதாவது LLM சரியாக தொடர்புடைய விவரங்களை அடையாளம் கொண்டும் சுருக்கமாகவும் வழங்கியுள்ளது என்பது தெளிவாகிறது.

#### மூல நோட்கள் கருத்துரை
மூல நோட்கள் சர்வதேச தடைகளைத் தொடர்ந்து ரஷ்யா சந்தித்த பொருளாதார விளைவுகள் பற்றிய விரிவான கணக்கைவழங்குகின்றன. உரை ரஷ்ய பங்கு சந்தை அதன் மதிப்பில் 40% இழந்தது என்றும் வர்த்தகம் நிறுத்தப்பட்டது என்றும் சிறப்பாகக் குறிப்பிடுகிறது. கூடுதலாக, இதில் ரூபிளின் மதிப்பு குறைதல் மற்றும் ரஷ்யாவின் பொருளாதாரத்தின் தளர்ச்சி போன்றப் பொது தனிமையாத் தாக்கங்களும்காணப்படுகின்றன. LLM பதில் இந்த நோடுகளிலிருந்து முக்கியமான புள்ளிகளை திறம்பட பிரித்துச் சொல்லி, கேள்வியின் கோரிக்கைகளைப் பொருத்தவரை பங்கு சந்தை தாக்கத்தின்மேல் கவனம் செலுத்தியுள்ளது.


இப்போது, Hybrid Search நல்ல ஆதாரமுள்ள பதிலை அளிக்காத ஒரு கேள்வியைப் பார்ப்போம்:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### ஹைபிரிட் தேடல்: LLM பதில் பகுப்பாய்வு
ஹைபிரிட் தேடல் எடுத்துக்காட்டில் உள்ள LLM பதில், வழங்கப்பட்ட சூழ்நிலை ரஷ்யாவின் உக்ரைனில் மேற்கொண்ட படையெடுப்பின் துல்லிய திகதி குறிப்பிடப்படவில்லை என்பதைக் குறிக்கிறது. இந்தப் பதில் LLM ஆதார ஆவணங்களில் உள்ள தகவல்களை பயன்படுத்துகிறதென காட்டுகிறது, ஆனால் உரையில் துல்லியமான விவரங்கள் இல்லை என்பதை ஒப்புக்கொள்கிறது.

அந்தப் பதில், சூழ்நிலையத்தில் ரஷ்யாவின் ஆகிரமிக்கையுடன் தொடர்புடைய நிகழ்வுகள் குறிப்பிடப்பட்டுள்ளன என்றதை அடையாளம் காண்பதில் துல்லியமானது; ஆனால் குறிப்பிட்ட படையெடுப்பு திகதியை குறிப்பிடவில்லை. இது வழங்கப்பட்ட தகவல்களைப் புரிந்து கொள்ளும் LLM-ன் திறனையும், உள்ளடக்கத்தில் உள்ள குறைபாடுகளை உணர்வதும் வெளிப்படுத்துகிறது. துல்லியத் திகதிக்காக பயனாளரை வெளிப்புற ஆதாரங்கள் அல்லது வரலாற்று பதிவுகளைத் தேடச் சொல்லுவதில் LLM திறமையாக ஊக்குவிக்கிறது, தகவல் முழுமையற்றபோது ஒரு அளவிலான எச்சரிக்கையாள்மையை காட்டுகிறது.

### ஆதார நோட்கள் பகுப்பாய்வு
ஹைபிரிட் தேடல் எடுத்துக்காட்டில் உள்ள ஆதார நோடுகள், உக்ரைனில் ரஷ்யாவின் நடவடிக்கைகளுக்கு அமெரிக்காவின் பதிலைக் குறித்து பேச்சின் மேற்கோள்களை உள்ளடக்கியுள்ளன. இந்த நோடுகள் பரபரப்பான புவிசார்-அரசியல் தாக்கம் மற்றும் படையெடுப்பிற்குப் பதிலளித்து அமெரிக்கா மற்றும் அதன் கூட்டாளர்களால் எடுத்த நடவடிக்கைகளை வலியுறுத்துகின்றன, ஆனால் அவை குறிப்பிட்ட படையெடுப்பு திகதியை குறிப்பிடவில்லை. இது சூழ்நிலையத்தில் துல்லியமான திகதி தகவல் இல்லை என்பதை சரியாக அடையாளம் காணும் LLM பதிலுடன் செய்யும் ஒத்திசைவை பிரதிபலிக்கிறது.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid w/Reranking: LLM Response Analysis
In the Hybrid w/Reranking example, the LLM response provides additional context by noting that the event occurred six days before the speech was given. This indicates that the LLM is able to infer the invasion date based on the timing of the speech, even though it still requires knowing the exact date of the speech for precision.

This response demonstrates an improved ability to use context clues to provide a more informative answer. It highlights the advantage of reranking, where the LLM can access and prioritize more relevant information to give a closer approximation of the desired detail (i.e., the invasion date).

### Source Nodes Analysis
The source nodes in this example include references to the timing of Russia's invasion, specifically mentioning that it occurred six days before the speech. While the exact date is still not explicitly stated, the nodes provide temporal context that allows the LLM to give a more nuanced response. The inclusion of this detail showcases how reranking can improve the LLM's ability to extract and infer information from the provided context, resulting in a more accurate and informative response.


**குறிப்பு:**
இந்த நோட்புக்கில், நாங்கள் NVIDIA API Catalog இலிருந்து NVIDIA NIM மைக்ரோசர்வீஸ்களை பயன்படுத்தினோம்.
மேலே உள்ள APIகள், `NVIDIA (llms)`, `NVIDIAEmbedding`, மற்றும் [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview). குறிப்பு: மேற்படி APIகள் சுய-ஹோஸ்ட் செய்யப்பட்ட மைக்ரோசர்வீஸ்களையும் ஆதரிக்கலாம். 

**உதாரணம்:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
மறுப்பு அறிவிப்பு:
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவையான [Co-op Translator](https://github.com/Azure/co-op-translator) பயன்படுத்தி மொழிபெயர்க்கப்பட்டது. நாங்கள் துல்லியத்திற்காக முயற்சித்தாலும், தானியங்கி மொழிபெயர்ப்புகளில் தவறுகள் அல்லது பிழைகள் இருக்கலாம் என்பதை தயவுசெய்து கருத்தில் கொள்ளவும். அசல் ஆவணம் அதன் சொந்த மொழியில் உள்ளதையே அதிகாரபூர்வ ஆதாரமாக கருத வேண்டும். முக்கியமான தகவல்களுக்காக, தொழில்முறை மனித மொழிபெயர்ப்பை பரிந்துரைக்கிறோம். இந்த மொழிபெயர்ப்பைப் பயன்படுத்தியதனால் ஏற்படும் எந்தவொரு தவறான புரிதலும் அல்லது தவறான பொருள் எடுத்துக்கொள்ளுதலுக்கும் நாங்கள் பொறுப்பேற்க மாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
